In [1]:
!pip install nest-asyncio
!pip install requests
!pip install beautifulsoup4
!pip install werkzeug
!pip install langchain
!pip install openai
!pip install faiss-cpu
!pip install pymupdf
! pip install langchain-openai
! pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 57.9 MB/s eta 0:00:00


In [174]:
import os
import nest_asyncio
import requests
from bs4 import BeautifulSoup
from werkzeug.utils import secure_filename
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyMuPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import re
from langchain.docstore.document import Document
from urllib.parse import urljoin
import mimetypes

In [8]:
# ตั้งค่า OpenAI API Key
openai_api_key =  "API KEY"
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1, max_tokens=2000, openai_api_key=openai_api_key)

# ตั้งค่าตัวแปลงเวกเตอร์

In [9]:
# ตั้งค่าตัวแปลงเวกเตอร์
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectors = None
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer")

<ipython-input-9-e29d5588498c>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or da

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-9-e29d5588498c>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer")


In [189]:
# ตั้งค่า Prompt Template สำหรับภาษาไทย
thai_prompt = ChatPromptTemplate.from_template(
    """
    คุณเป็นแชทบอทที่ช่วยตอบคำถามจากเอกสารที่ให้มาโดยตรง
    โปรดใช้เฉพาะข้อมูลจากเอกสารที่ให้มาในการตอบคำถาม
    หากไม่พบข้อมูลในเอกสาร กรุณาตอบว่า "ไม่พบข้อมูลดังกล่าวในเอกสาร"
    แต่ถ้ามันมีในเอกสารแต่ต้องใช้การวิเคราะห์ ให้ตอบว่า "ไม่ได้มีการบอกถึงในเอกสารโดยตรงแต่......"
    โดยคำตอบควรไม่สั้นเกินไปและยาวเกินไป
    ตอบคำถามเป็นภาษาไทยเท่านั้น ห้ามมีภาษาจีน

    ### **บริบทของเอกสาร:**
    {context}

    ### **คำถาม:**
    {question}

    ### **คำตอบ:**
    """
)

In [25]:
# ตั้งค่า Prompt Template สำหรับภาษาอังกฤษ
english_prompt = ChatPromptTemplate.from_template(
    """
    You are a chatbot that helps answer questions directly from the provided documents.
    Please use only information from the provided documents to answer the question.
    If the information is not found in the document, please answer "This information is not found in the document".
    If it's in the document but requires analysis, answer "It's not directly mentioned in the document, but..."
    The answer should not be too short or too long.
    Answer in English only.

    ### **Document Context:**
    {context}

    ### **Question:**
    {question}

    ### **Answer:**
    """
)

In [26]:
def clean_text(text):
    # ลบอักขระที่ไม่ใช่ภาษาไทย, อังกฤษ, ช่องว่าง และขึ้นบรรทัดใหม่
    text = re.sub(r'[^\u0E00-\u0E7F\u0020-\u007E\n]', '', text)
    # ลบบรรทัดที่มีตัวอักษรเพียง 1 ตัว (ไม่นับช่องว่าง)
    text = '\n'.join([line for line in text.split('\n') if len(line.strip()) > 1])
    return text

In [188]:
# ฟังก์ชัน Paraphrase คำตอบ
def paraphrase_answer(raw_answer, language="auto"):
    try:
        if not isinstance(raw_answer, str):
            raw_answer = str(raw_answer)

        if language == "th":
            paraphrase_prompt = f"""
            กรุณาปรับคำตอบให้อ่านง่ายและเป็นธรรมชาติ
            หลีกเลี่ยงการใช้คำหรือสัญลักษณ์ที่ผิดปกติ
            ตอบให้อยู่ในรูปแบบที่เหมือนมนุษย์พูด โดยยังคงข้อมูลที่ถูกต้องตามต้นฉบับ

            ตอบเป็นภาษาไทยเท่านั้น แม้ว่าคำตอบเดิมจะเป็นภาษาอังกฤษก็ตาม ห้ามมีภาษาจีน

            ### คำตอบเดิม:
            {raw_answer}

            ### คำตอบที่ปรับปรุงแล้ว:
            """
        elif language == "en":
            paraphrase_prompt = f"""
            Please adjust the answer to be easy to read and natural.
            Avoid using unusual words or symbols.
            Answer in a human-like format while maintaining the accuracy of the original information.

            Answer in English only, even if the original answer is in Thai.ห้ามมีภาษาจีน

            ### Original Answer:
            {raw_answer}

            ### Improved Answer:
            """
        else:
            paraphrase_prompt = f"""
            กรุณาปรับคำตอบให้อ่านง่ายและเป็นธรรมชาติ
            หลีกเลี่ยงการใช้คำหรือสัญลักษณ์ที่ผิดปกติ
            ตอบให้อยู่ในรูปแบบที่เหมือนมนุษย์พูด โดยยังคงข้อมูลที่ถูกต้องตามต้นฉบับ

            ตรวจสอบภาษาของคำตอบเดิม (ไทยหรืออังกฤษ) และใช้ภาษาเดียวกันในการตอบกลับ
            ถ้าคำตอบเดิมเป็นภาษาอังกฤษ ให้ตอบเป็นภาษาอังกฤษ ถ้าคำตอบเดิมเป็นภาษาไทย ให้ตอบเป็นภาษาไทย ห้ามมีภาษาจีน

            ### คำตอบเดิม:
            {raw_answer}

            ### คำตอบที่ปรับปรุงแล้ว:
            """

        response = llm.invoke(paraphrase_prompt)
        return response.content if hasattr(response, 'content') else str(response)
    except Exception as e:
        print(f"⚠️ Paraphrase Error: {e}")
        return raw_answer

In [129]:
# ฟังก์ชันดึงข้อมูลจาก URL
def fetch_page_text(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        text = soup.get_text(separator="\n", strip=True)

        return text
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching {url}: {e}")
        return None


In [31]:
# ฟังก์ชันโหลดเอกสาร PDF พร้อม preprocess
def load_document(file_path):
    global vectors
    loader = PyMuPDFLoader(file_path)
    documents = loader.load()

    for doc in documents:
        doc.page_content = clean_text(doc.page_content)

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.split_documents(documents)
    vectors = FAISS.from_documents(docs, embeddings)
    return vectors

In [164]:
def load_url(url):
    global vectors
    text = fetch_page_text(url)
    if not text:
        return None

    # ✅ ทำความสะอาดข้อความอีกครั้งก่อนสร้างเวกเตอร์
    cleaned_text = clean_text(text)

    # สร้างเอกสารจากข้อความที่ดึงมา
    documents = [Document(page_content=cleaned_text)]

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.split_documents(documents)
    vectors = FAISS.from_documents(docs, embeddings)
    return vectors


In [145]:
# ฟังก์ชันถามคำถาม
def ask_question(question, language="auto"):
    global vectors, memory
    if vectors is None:
        return "กรุณาอัพโหลดไฟล์ก่อนเริ่มถามคำถาม"

    if language == "th":
        prompt_template = thai_prompt
    elif language == "en":
        prompt_template = english_prompt
    else:
        prompt_template = thai_prompt  # ค่าเริ่มต้นเป็นภาษาไทย

    qa_chain = ConversationalRetrievalChain.from_llm(
        llm,
        retriever=vectors.as_retriever(search_type="similarity", search_kwargs={"k": 4}),
        memory=memory,
        return_source_documents=True,
        output_key="answer",
        combine_docs_chain_kwargs={"prompt": prompt_template}
    )

    response = qa_chain({"question": question})
    raw_answer = response.get("answer", "ไม่พบคำตอบที่เกี่ยวข้อง")
    paraphrased_answer = paraphrase_answer(raw_answer, language)

    memory.chat_memory.add_user_message(question)
    memory.chat_memory.add_ai_message(paraphrased_answer)

    return paraphrased_answer

In [81]:
# ฟังก์ชันเคลียร์ประวัติการสนทนา
def clear_history():
    global vectors, memory
    vectors = None
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer")
    return "ล้างประวัติการสนทนาและข้อมูลเรียบร้อยแล้ว"

# ทดสอบไฟล์ pdf

In [120]:
file_path = "/content/press-release-krungsri-wisesight-series-b-th.pdf"  # ใส่พาธของไฟล์ PDF ที่ต้องการโหลด
load_document(file_path)

In [121]:
question = "TechMatrix คืออะไร?"  # ใส่คำถามที่ต้องการถามเกี่ยวกับเอกสาร
answer = ask_question(question, language="en")  # ใช้ "th" สำหรับภาษาไทย หรือ "en" สำหรับภาษาอังกฤษ

In [122]:
print(answer)

TechMatrix is a company in Japan that offers IT solutions using advanced technology to help businesses grow. They are known for their work in the IT solutions industry and are focused on creating new ideas to enhance their CRM business. Recently, they teamed up with WiseSight to develop a complete CRM solution for businesses and to expand their services to the ASEAN region.


In [123]:
question = "FastSeries คืออะไร"  # ใส่คำถามที่ต้องการถามเกี่ยวกับเอกสาร
answer = ask_question(question, language="th")

In [124]:
print(answer)

FastSeries เป็นระบบบริหารงานลูกค้าสัมพันธ์บนคลาวด์ที่ได้รับการยอมรับอย่างแพร่หลายในประเทศญี่ปุ่น ซึ่งเน้นการเพิ่มมูลค่าให้องค์กรโดยมอบประสบการณ์ที่ดีที่สุดให้แก่ลูกค้า (CX) โดยช่วยให้บริษัทเข้าใจความรู้สึกของลูกค้าต่อผลิตภัณฑ์หรือบริการที่ได้รับ นอกจากนี้ยังช่วยเพิ่มการเติบโตของธุรกิจ FastSeries ในประเทศไทยและประเทศในอาเซียน ดังนั้น FastSeries เป็นส่วนสำคัญที่ช่วยสร้างประสบการณ์ที่ดีให้แก่ลูกค้า และเพิ่มความพึงพอใจในการให้บริการของตนเอง นอกจากนี้ยังเป็นกำลังขับเคลื่อนสำคัญที่ช่วยให้บริษัทประสบความสำเร็จและเติบโตต่อไปในอนาคตได้


In [125]:
for message in memory.chat_memory.messages:
    print(message.type, ":", message.content)

human : TechMatrix คืออะไร?
ai : TechMatrix is a company that provides IT solutions incorporating cutting-edge technology to transform businesses. They are a major player in the IT solutions industry in Japan and focus on developing innovations to expand their CRM business. They have partnered with WiseSight to create a comprehensive CRM solution to meet business needs and expand into the ASEAN region.
human : TechMatrix คืออะไร?
ai : TechMatrix is a company in Japan that offers IT solutions using advanced technology to help businesses grow. They are known for their work in the IT solutions industry and are focused on creating new ideas to enhance their CRM business. Recently, they teamed up with WiseSight to develop a complete CRM solution for businesses and to expand their services to the ASEAN region.
human : FastSeries คืออะไร
ai : FastSeries เป็นหนึ่งในระบบบริหารงานลูกค้าสัมพันธ์บนคลาวด์ที่ได้รับการยอมรับอย่างแพร่หลายโดยบริษัทชั้นนำในประเทศญี่ปุ่น ซึ่งมุ่งเน้นที่การเพิ่มมูลค่าให้อ

In [168]:
clear_message = clear_history()
print(clear_message)

ล้างประวัติการสนทนาและข้อมูลเรียบร้อยแล้ว


# ทดสอบ https://wisesight.com/about-us/

In [155]:
url = "https://wisesight.com/about-us/"
vectors = load_url(url)

In [156]:
question = "Solution ของ wisesight มีอะไรบ้าง?"  # ใส่คำถามที่ต้องการถามเกี่ยวกับเอกสาร
answer = ask_question(question, language="en")  # ใช้ "th" สำหรับภาษาไทย หรือ "en" สำหรับภาษาอังกฤษ

In [157]:
print(answer)

Wisesight's solution includes Zocial Eye, Influencer Directory, Social Metric, Omni Channel Solution, Warroom, Command Center, Service, Research, Data Consulting, Chatbot Service, and MONITORING & ALERT. These tools are used to analyze data on social media for Wisesight's clients and partners to take care of their customers effectively.


In [158]:
question = "WISESIGHT คือบริษัทอะไร"  # ใส่คำถามที่ต้องการถามเกี่ยวกับเอกสาร
answer = ask_question(question, language="en")  # ใช้ "th" สำหรับภาษาไทย หรือ "en" สำหรับภาษาอังกฤษ

In [159]:
print(answer)

WISESIGHT is a company that specializes in social media analytics services. They offer various services such as Zocial Eye, Influencer Directory, Social Metric, Omni Channel Solution, Warroom, Command Center, Service, Research, Data Consulting, Chatbot Service, MONITORING & ALERT, and others as specified in their documentation. They are experts in using social media data to help clients better manage their customers and are well-known for their social media analytics expertise in Thailand.


In [160]:
for message in memory.chat_memory.messages:
    print(message.type, ":", message.content)

human : Solution ของ wisesight มีอะไรบ้าง?
ai : Solution ของ Wisesight ประกอบด้วย Zocial Eye, Influencer Directory, Social Metric, Omni Channel Solution, Warroom, Command Center, Service, Research, Data Consulting, Chatbot Service, และ MONITORING & ALERT ซึ่งเป็นเครื่องมือในการวิเคราะห์ข้อมูลบนโซเชียลมีเดียให้กับลูกค้าและพาร์ทเนอร์ของ Wisesight ในการดูแลลูกค้าของตนอย่างดี.
human : Solution ของ wisesight มีอะไรบ้าง?
ai : Wisesight's solution includes Zocial Eye, Influencer Directory, Social Metric, Omni Channel Solution, Warroom, Command Center, Service, Research, Data Consulting, Chatbot Service, and MONITORING & ALERT. These tools are used to analyze data on social media for Wisesight's clients and partners to take care of their customers effectively.
human : WISESIGHT คือบริษัทอะไร
ai : WISESIGHT เป็นบริษัทที่ให้บริการวิเคราะห์ข้อมูลสื่อสังคม (Social Media Analytics) และมีบริการต่างๆ เช่น Zocial Eye, Influencer Directory, Social Metric, Omni Channel Solution, Warroom, Command Center,

In [161]:
clear_message = clear_history()
print(clear_message)

ล้างประวัติการสนทนาและข้อมูลเรียบร้อยแล้ว


# ทดสอบ https://onlinemedia.idea2mobile.com/?p=6570

In [169]:
url = "https://onlinemedia.idea2mobile.com/?p=6570"
vectors = load_url(url)

In [170]:
question = "Wisesight ทุ่มเงินพัฒนาเทคโนโลยีอะไรอะไร?"  # ใส่คำถามที่ต้องการถามเกี่ยวกับเอกสาร
answer = ask_question(question, language="th")  # ใช้ "th" สำหรับภาษาไทย หรือ "en" สำหรับภาษาอังกฤษ

In [171]:
print(answer)

Wisesight ลงทุนในการพัฒนาเทคโนโลยี A.I. (Artificial Intelligence) เพื่อช่วยวิเคราะห์ข้อมูลจาก Social Listening โดยใช้ KIRIN A.I. Engine เพื่อหา Insight จากข้อมูลบน Social Media ในปี 2023 ซึ่งเป็นส่วนสำคัญในการพัฒนาเทคโนโลยีในอุตสาหกรรม Social Listening ในอนาคต การใช้ A.I. เป็นเทคโนโลยีที่ Wisesight ให้ความสำคัญในการพัฒนาเทคโนโลยีของตน และเป็นส่วนหนึ่งของ ZOCIAL EYE ในการให้บริการวิเคราะห์ข้อมูลจาก Social Media ให้กับลูกค้าในอุตสาหกรรมต่าง ๆ ในปัจจุบันและอนาคต ดังนั้นการพัฒนาเทคโนโลยี A.I. เป็นสำคัญในยุคใหม่ของ Social Listening และการวิเคราะห์ข้อมูลจาก Social Media ในอนาคต ตามที่เอกสารกล่าวถึง


In [172]:
question = "Package SMEs ราคาเริ่มต้นเท่าไหร่"  # ใส่คำถามที่ต้องการถามเกี่ยวกับเอกสาร
answer = ask_question(question, language="en")  # ใช้ "th" สำหรับภาษาไทย หรือ "en" สำหรับภาษาอังกฤษ

In [173]:
print(answer)

The starting price for Package SMEs is 3,000 baht per month, and there is a free trial available for 7 days as stated in the document. This package allows access to KIRN A.I. at the mentioned monthly price.


# ลองสร้าง chatbot ของลิ้งทุกอันที่ถูกติดในเว็ปบริษัท wisesight

In [181]:
def get_all_urls(base_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
    }

    try:
        response = requests.get(base_url, headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        urls = set()

        for link in soup.find_all('a', href=True):
            full_url = urljoin(base_url, link['href'])

            # ตรวจสอบประเภทไฟล์จากนามสกุล URL
            mime_type, _ = mimetypes.guess_type(full_url)
            if mime_type:
                if mime_type.startswith("image/"):
                    print(f"🖼️ Skipping image file: {full_url}")
                    continue
                if mime_type in ["application/zip", "application/x-rar-compressed", "application/x-tar", "application/gzip"]:
                    print(f"📦 Skipping archive file: {full_url}")
                    continue

            urls.add(full_url)

        return list(urls)

    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return []

def fetch_page_text(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        text = soup.get_text(separator="\n", strip=True)

        return text

    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching {url}: {e}")
        return None


def save_to_txt(filename, text):
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(text)

# ตัวอย่างการใช้งาน
base_url = "https://wisesight.com/about-us/"  # เปลี่ยนเป็น URL ที่ต้องการดึงข้อมูล
urls = get_all_urls(base_url)

all_text = ""
for url in urls:
    print(f"Fetching content from: {url}")
    content = fetch_page_text(url)
    if content:
        all_text += f"### URL: {url} ###\n{content}\n\n"

# บันทึกเป็นไฟล์ .txt
save_to_txt("scraped_content.txt", all_text)
print("✅ ข้อมูลถูกบันทึกลงไฟล์ scraped_content.txt")

🖼️ Skipping image file: https://wisesight.com/wp-content/uploads/2021/04/people-puttasak-tantisuttivet-3.jpg
🖼️ Skipping image file: https://wisesight.com/wp-content/uploads/2021/04/people-puttasak-tantisuttivet-2.jpg
🖼️ Skipping image file: https://wisesight.com/wp-content/uploads/2021/04/people-puttasak-tantisuttivet-1.jpg
🖼️ Skipping image file: https://wisesight.com/wp-content/uploads/2021/04/people-thaveechok-chokpongudomchai-3.jpg
🖼️ Skipping image file: https://wisesight.com/wp-content/uploads/2021/04/people-thaveechok-chokpongudomchai-2.jpg
🖼️ Skipping image file: https://wisesight.com/wp-content/uploads/2021/04/people-thaveechok-chokpongudomchai-1.jpg
🖼️ Skipping image file: https://wisesight.com/wp-content/uploads/2021/10/Wisesight-Logo-05-e1633680220324.png
📦 Skipping archive file: https://wisesight.com/wp-content/uploads/2021/10/manual-ci-wisesight.zip
🖼️ Skipping image file: https://wisesight.com/wp-content/uploads/2021/10/Wisesight-Logo-04.png
📦 Skipping archive file: htt

In [182]:
# ฟังก์ชันโหลดไฟล์ TXT และสร้างเวกเตอร์
def load_text_file(file_path):
    global vectors
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()

        # ✅ ทำความสะอาดข้อความก่อนสร้างเวกเตอร์
        cleaned_text = clean_text(text)

        # สร้างเอกสารจากข้อความที่อ่านมา
        documents = [Document(page_content=cleaned_text)]

        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        docs = text_splitter.split_documents(documents)
        vectors = FAISS.from_documents(docs, embeddings)
        return vectors
    except Exception as e:
        print(f"❌ Error loading TXT file: {e}")
        return None

In [183]:
file_path = "/content/scraped_content.txt"  # ใส่พาธของไฟล์ PDF ที่ต้องการโหลด
load_text_file(file_path)

In [194]:
question = "Scene Recognition คืออะไร?"  # ใส่คำถามที่ต้องการถามเกี่ยวกับเอกสาร
answer = ask_question(question, language="th")  # ใช้ "th" สำหรับภาษาไทย หรือ "en" สำหรับภาษาอังกฤษ

In [195]:
print(answer)

การระบุฉาก (Scene Recognition) คือกระบวนการที่ใช้เทคโนโลยีเพื่อระบุหรือจำแนกแนวทางของภาพหรือวิดีโอว่าเป็นฉากอะไร เช่น ป่า, ชายหาด, สนามกอล์ฟ, ห้องน้ำ, หรืออื่นๆ โดยใช้ข้อมูลที่ได้จากการวิเคราะห์ภาพหรือวิดีโอเพื่อระบุลักษณะเฉพาะของแต่ละฉากในภาพหรือวิดีโอนั้น การระบุฉากสามารถช่วยในการจำแนกประเภทของภาพหรือวิดีโอได้อย่างถูกต้องและรวดเร็ว และมีการนำไปใช้ในหลากหลายงาน เช่น การจัดการภาพ, การค้นหาข้อมูล, หรือการตรวจจับวัตถุในภาพได้ดีมากขึ้น
